In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score as auc

In [ ]:
%load_ext autoreload
%autoreload 2
from mriqc_learn.datasets import load_dataset
from mriqc_learn.models import preprocess as pp
from mriqc_learn.models.production import init_pipeline
from mriqc_learn.model_selection import split

## Load some data
We first load the CHUV100 dataset

In [ ]:
(train_x, train_y), (_, _) = load_dataset(
    dataset="chuv100",
    split_strategy="none",
    first_iqm="centroid",
)
#train_x["site"] = train_y.site
train_x["sub_ses"] = train_y.sub_ses

Let's pick the ratings from "rater_3" and binarize the three categories into only two.
We can also see that the dataset is unbalanced.

In [ ]:
train_y["rating_avg"] = train_y[["rater_1","rater_2"]].values.mean(axis=1)
train_y[["rating_avg"]]

Let's print out a pretty view of the data table:

In [ ]:
train_x

## Visualization and analysis of data

In [ ]:
from mriqc_learn.viz import metrics
from mriqc_learn.models.preprocess import SiteRobustScaler

In [ ]:
f = metrics.plot_batches(train_x,group_by="sub_ses")

In [ ]:
scaled_x = SiteRobustScaler(unit_variance=True, groupby="sub_ses").fit_transform(train_x)
fig2 = metrics.plot_batches(scaled_x, group_by="sub_ses")

In [ ]:
import seaborn as sns

In [ ]:
def find_largest_correlation(corrmat, threshold=0.95, skip_nan=False, verbose=False):
    # Select upper triangle of correlation matrix
    if skip_nan:
        keys_no_nan = [k for k in corrmat.columns if "_nan" not in k]
        corrmat = corrmat.loc[keys_no_nan][keys_no_nan]
    upper = corrmat.where(np.triu(np.ones(corrmat.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(abs(upper[column]) > threshold)]
    if verbose:
        for col in sorted(to_drop):
            df_col = upper[col]
            print(f"{col} correlated with:")
            for k,v in df_col[abs(df_col) > threshold].items():
                print(f"\t{k} ({v:.2f})")
    return to_drop

In [ ]:
# Re-order columns: features first, whether there is a nan second
metrics = train_x.copy()
metrics = metrics.drop("sub_ses",axis=1)
cols_nan = [col for col in metrics.columns if "_nan" in col]
cols_rest = [col for col in metrics.columns if "_nan" not in col]
metrics = metrics[cols_rest + cols_nan]

cst_cols = metrics.columns[(metrics.var() == 0).values].tolist()
print("Dropped columns (constant):", cst_cols)
metrics = metrics.drop(cst_cols, axis=1)
to_drop = find_largest_correlation(metrics.corr(), threshold=0.92, skip_nan=False)
print("Redundant metrics:", to_drop)
metrics = metrics.drop(to_drop, axis=1)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_lower_tri_heatmap(df, output="cooc_matrix.png"):
    mask = np.zeros_like(df, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Want diagonal elements as well
    mask[np.diag_indices_from(mask)] = False

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns_plot = sns.heatmap(df, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
    f.show()
    # save to file
    #fig = sns_plot.get_figure()
    #fig.savefig(output)
get_lower_tri_heatmap(metrics.corr())

In [ ]:
metrics_sets = {f"set{i+1}": list(metrics.columns[i*10:i*10+9]) for i in range(5)}
for metrics_set in metrics_sets.values():
     sns.pairplot(metrics[metrics_set])

In [ ]:
metrics.plot_histogram(train_x, scaled_x, groupby="sub_ses",metric="centroid");

In [ ]:
metrics.plot_histogram(train_x, scaled_x, groupby="sub_ses",metric="dl_slice_iqa_full");

## Cross-validation of the default classifier
Let's cross-validate the performance of our classifier using a Leave-one-site-out strategy.

In [ ]:
# Define a splitting strategy
from mriqc_learn.model_selection import split as sp
outer_cv = sp.LeavePSitesOut(1, colname="sub_ses", robust=True, shuffle=True)

We can now feed the model into the cross-validation loop:

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor as GBR
model = GBR()

x = train_x.drop("sub_ses",axis=1)
x = x.drop(cst_cols+to_drop,axis=1)
y = np.array(train_y[["rating_avg"]]).flatten()
cv_score = []
for train, test in outer_cv.split(train_x, y):
    xtr, ytr = x.loc[train], y[train]
    xte, yte = x.loc[test], y[test]
    res = model.fit(xtr, ytr)
    ypred = res.predict(xte).flatten()
    ypred = np.clip(ypred, 1,4)
    mae = mean_absolute_error(yte, ypred)
    cv_score += [mae]
print(np.mean(cv_score))

In [ ]:
sub_ses = list(train_x["sub_ses"])
group_og = []
group_dict = {}
curr_group = 0
for idx in sub_ses:
    if idx in group_dict.keys():
        group_og.append(idx)
    else:
        group_dict[idx] = curr_group
        group_og.append(idx)
        curr_group+=1
group = group_og

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import KFold

model = GBR()

x = train_x.drop("sub_ses",axis=1)
x = x.drop(cst_cols+to_drop,axis=1)
y = np.array(train_y[["rating_avg"]]).ravel()
mae, r2 = [], []
kf = KFold(n_splits=5)
#groups = train_y[["sub_ses"]]
split = kf.split(train_x, train_y[["rating_avg"]],group)
for train, test in split:
    xtr, ytr = x.loc[train], y[train]
    xte, yte = x.loc[test], y[test]
    g = np.array(group)
    print(np.unique(g[train]), np.unique(g[test]))
    clf = model.fit(xtr, ytr)
    ypred = model.predict(xte).flatten()
    #ypred = np.clip(ypred, 1,4)
    mae_, r2_ = mean_absolute_error(yte, ypred), r2_score(yte, ypred)
    mae += [mae_]
    r2 += [r2_]
print(f"{clf}: MAE={np.mean(mae):.3f}, r2={np.mean(r2):.3f}")

In [ ]:
def z_score(x):
    return (x - x.mean())/(x.std()+1e-8)

norm_x = train_x.drop("sub_ses",axis=1).apply(z_score)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
cv_score = cross_val_score(
    GradientBoostingRegressor(),
    X=train_x.drop("sub_ses",axis=1),
    y=np.array(train_y[["rating_avg"]]).flatten(),
    groups=train_x[["sub_ses"]],
    cv=KFold(n_splits=5, shuffle=False),
    scoring="r2",   
)

In [ ]:
cv_score.mean()

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.feature_selection import RFE
random_state = 432
nsplits=5
nrepeats = 1
#rkf = RepeatedKFold(n_splits=nsplits, n_repeats=10,random_state=random_state)
rkf = KFold(n_splits=nsplits, shuffle=True)
models = [
    GradientBoostingRegressor()
]
y = np.array(train_y[["rating_avg"]]).ravel()
for Xstr, x in {"X":train_x, "X_norm": scaled_x, "X_norm2":norm_x}.items():
    print(f"\n{Xstr}")
    if "sub_ses" in  x.columns:
        print("SUB_SES FOUND:")
        group2 = x["sub_ses"]
        x = x.drop("sub_ses", axis=1)
    else:
        group2 = group
        print(len(group2))
    for clf in models:
        out_tot[Xstr][clf] = []
        split = rkf.split(x,y, group2)
        mae_tot = []
        r2_tot = []
        
        for i, (train, test) in enumerate(split):
            xtr,ytr=x.loc[train], y[train]
            xte,yte=x.loc[test], y[test]
            
            reg = clf.fit(xtr, ytr)
            ypred = clf.predict(xte).flatten()
            ypred = np.clip(ypred, 1,4)
            
            mae, r2 = mean_absolute_error(yte, ypred), r2_score(yte, ypred)
            mae_tot.append(mae)
            r2_tot.append(r2)
        #print(mae_tot, r2_tot)
        print(f"{clf}: MAE={np.mean(mae_tot):.3f}+-{np.std(mae_tot):.3f}, r2={np.mean(r2_tot):.3f}")

In [ ]:
SUB_SES FOUND:
GradientBoostingRegressor(): MAE=0.350, r2=0.562

X_norm
SUB_SES FOUND:
GradientBoostingRegressor(): MAE=0.433, r2=0.360

X_norm2
GradientBoostingRegressor(): MAE=0.348, r2=0.564


X
LinearRegression(): MAE=0.664, r2=-0.665
	Spearman=0.472 0.0014
Ridge(): MAE=0.401, r2=0.338
	Spearman=0.707 0.0000
DecisionTreeRegressor(): MAE=0.444, r2=0.229
	Spearman=0.550 0.0004
GradientBoostingRegressor(): MAE=0.357, r2=0.535
	Spearman=0.733 0.0000
GradientBoostingRegressor(loss='absolute_error'): MAE=0.345, r2=0.592
	Spearman=0.779 0.0000

X_norm
LinearRegression(): MAE=0.979, r2=-2.159
	Spearman=0.087 0.5626
Ridge(): MAE=0.832, r2=-1.342
	Spearman=0.072 0.2945
DecisionTreeRegressor(): MAE=0.583, r2=-0.287
	Spearman=0.336 0.0260
GradientBoostingRegressor(): MAE=0.437, r2=0.345
	Spearman=0.526 0.0000
GradientBoostingRegressor(loss='absolute_error'): MAE=0.475, r2=0.284
	Spearman=0.473 0.0003

X_norm2
LinearRegression(): MAE=0.796, r2=-1.343
	Spearman=0.290 0.1752
Ridge(): MAE=0.369, r2=0.474
	Spearman=0.722 0.0000
DecisionTreeRegressor(): MAE=0.430, r2=0.310
	Spearman=0.619 0.0000
GradientBoostingRegressor(): MAE=0.357, r2=0.534
	Spearman=0.731 0.0000
GradientBoostingRegressor(loss='absolute_error'): MAE=0.350, r2=0.575
	Spearman=0.773 0.0000

In [ ]:
cv_score = cross_val_score(
    init_pipeline(
        model = "gradient_boosting_regression_l1",
        drop_ft = False,
        use_classifier = False,
        groupby = "sub_ses",
    ),
    X=train_x,
    y=np.array(train_y[["rating_avg"]]).flatten(),
    cv=outer_cv,
    scoring="r2",
    error_score='raise'
)

After one or two minutes, the scores have been caculated for each of the 14 folds our splitter created.
The average performance is AUC=0.885.

In [ ]:
[f"{x:.3f}" for x in cv_score]

In [ ]:
print(cv_score)
cv_score.mean()

In [ ]:
custom_cv_score = {}
for train_index, (site, test_index) in outer_cv.split(train_x, y=train_y, return_key=True):
    # Validate on test fold
    print(f"Validating on left-out site ({site})...")
    model_split = init_pipeline()
    model_split = model_split.fit(train_x.iloc[train_index], train_y[train_index])
    custom_cv_score[site] = auc(train_y[test_index], model_split.predict(train_x.iloc[test_index]))

In [ ]:
print(custom_cv_score)
np.mean(list(custom_cv_score.values()))

We now train the model on all available training data:

In [ ]:
model = init_pipeline().fit(
    X=train_x,
    y=train_y,
)    

We can easily see the effects of overfitting by evaluating the classifier on the same folds we used for cross-validation.

In [ ]:
overfit_cv_score = {}
for train_index, (site, test_index) in outer_cv.split(train_x, y=train_y, return_key=True):
    print(f"Validating on left-out site ({site})...")
    overfit_cv_score[site] = auc(train_y[test_index], model.predict(train_x.iloc[test_index]))

In [ ]:
print([overfit_cv_score[s] - custom_cv_score[s] for s in overfit_cv_score.keys()])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(train_y, model.predict(train_x)))

## Evaluating on held-out dataset
We first load the held-out dataset in, and evaluate:

In [ ]:
(test_x, test_y), (_, _) = load_dataset("ds030", split_strategy="none")
test_x["site"] = test_y.site
test_x

In [ ]:
has_ghost = test_y.has_ghost.values.astype(bool)
test_y = test_y[["rater_1"]].values.squeeze().astype(int)
print(f"Discard={100 * (test_y == -1).sum() / len(test_y)}")
print(f"Doubtful={100 * (test_y == 0).sum() / len(test_y)}")
print(f"Accept={100 * (test_y == 1).sum() / len(test_y)}")
test_y[test_y < 1] = 0

In [ ]:
auc(test_y, model.predict(test_x))

In [ ]:
auc(test_y[~has_ghost], model.predict(test_x[~has_ghost]))

In [ ]:
print(classification_report(test_y, model.predict(test_x)))

In [ ]:
print(classification_report(test_y[~has_ghost], model.predict(test_x[~has_ghost])))

## Nested cross-validation

In [ ]:
p_grid = [{
    "scale__unit_variance": [True, False],
    "scale__with_centering": [True, False],
    "site_pred__disable": [False, True],
    "winnow__disable": [False, True],
    "svc__kernel": ["rbf"],
    "svc__C": [10],
    "svc__gamma": [0.1],
}]

In [ ]:
# Nested CV with parameter optimization
inner_cv = split.LeavePSitesOut(1, robust=True)
inner_cv.get_n_splits(X=train_x, y=train_y)

clf = GridSearchCV(
    estimator=pipe,
    param_grid=p_grid,
    cv=inner_cv,
    n_jobs=30,
    scoring="roc_auc",
)
# clf.fit(train_x, y=train_y)

In [ ]:
nested_score = cross_val_score(
    clf,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    verbose=10,
    n_jobs=16,
)
nested_score.mean()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_